**Imports**

In [1]:
import pandas as pd
import plotly.express as px

from os import path

**Utilities**

In [2]:
def filter(lower: float, upper: float, df: pd.DataFrame) -> pd.DataFrame:
    diff = (df < upper) & (df > lower)
    return df.drop(columns=df.columns[diff.all()])

def relative(symbol: str, df: pd.DataFrame, bm: pd.DataFrame) -> pd.DataFrame:
    rel = df.reindex(bm.index, method="bfill")
    rel[symbol] = rel["close"].div(bm["close"] * rel["close"][0]) * bm["close"][0]
    return rel

def calc_relative(symbol: str, data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    bm = data[symbol]
    rel = pd.DataFrame(index = bm.index)
    buffer = []
    for symbol, kline in data.items():
        try:
            df = relative(symbol, kline, bm)
            buffer.append(df[symbol])
        except TypeError:
            print(f"Error while processing {symbol}")
    rel = pd.concat(buffer, axis=1)
    return rel

**Load Data**

In [3]:
data_path = path.join("..", "data")

usdt_markets = pd.read_csv(path.join(data_path, "USDT_markets.csv"))
btc_markets = pd.read_csv(path.join(data_path, "BTC_markets.csv"))

usdt_minutly = {}
usdt_hourly = {}

for market in usdt_markets["symbol"]:
    usdt_minutly[market] = pd.read_csv(path.join(data_path, f"{market}_minutly.csv"), index_col="time")
    usdt_hourly[market] = pd.read_csv(path.join(data_path, f"{market}_hourly.csv"), index_col="time")

**Calculate Relative Performance**

In [4]:
intra = {k: v[-96:] for k, v in usdt_minutly.items()}
day = {k: v[-24:] for k, v in usdt_hourly.items()}
week = {k: v[(-24*7):] for k, v in usdt_hourly.items()}

rel_intra = calc_relative("BTCUSDT", intra)
rel_day = calc_relative("BTCUSDT", day)
rel_week = calc_relative("BTCUSDT", week)
rel_month = calc_relative("BTCUSDT", usdt_hourly)

rel_intra = filter(0.95, 1.05, rel_intra)
rel_day = filter(0.90, 1.10, rel_day)
rel_week = filter(0.85, 1.15, rel_week)
rel_month = filter(0.70, 1.30, rel_month)

**Plotting**

In [ ]:
fig_i = px.line(rel_intra, x = rel_intra.index, y = rel_intra.columns)
fig_d = px.line(rel_day, x = rel_day.index, y = rel_day.columns)
fig_w = px.line(rel_week, x = rel_week.index, y = rel_week.columns)
fig_m = px.line(rel_month, x = rel_month.index, y = rel_month.columns)

fig_i.show()
fig_d.show()
fig_w.show()
fig_m.show()

In [ ]:
a = rel_intra.mean().dropna().sort_values()
b = rel_day.mean().dropna().sort_values()
c = rel_week.mean().dropna().sort_values()
d = rel_month.mean().dropna().sort_values()

fig_i = px.bar(a, x = a.index, y = a-1)
fig_d = px.bar(b, x = b.index, y = b-1)
fig_w = px.bar(c, x = c.index, y = c-1)
fig_m = px.bar(d, x = d.index, y = d-1)

fig_i.show()
fig_d.show()
fig_w.show()
fig_m.show()